# 1. 파일 만들기

In [4]:
import os
import csv

file_path = "hand_data.csv"

# file_path가 없으면 직접 만들기
if not os.path.exists(file_path):
    with open(file_path, "w") as file:
        writer = csv.writer(file)
        writer.writerow([1, 2, 3, 4, 5])
        # csv 파일 확인해보기

In [5]:
# 파일에 데이터 추가하기
with open(file_path, "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([5, 4, 3, 2, 1])

In [ ]:
# hand landmark 옵션
# (New) 저장할 파일명 넣기 file_path
# (New) 만약에 파일경로가 없으면 새로 만들기
# 웹캠
#   # 카메라 감지
#   # 좌우 반전
#   # 손 그리기 준비
#   # 손 감지
#   # Hand Landmark 추출
#       # 손 하나의 Hand Landmark 추출
#       # 저장 데이터 만들기, 포인트 그리기
#       # (New) 1누르면 rock, 2누르면 scissors, 3누르면 paper 그리고 데이터 저장
#   # 화면 띄우기
#   # ESC 누르면 종료
# 마무리

# 2. 데이터 학습

## 1) 데이터 불러오기

```
uv add pandas
```

In [35]:
import pandas as pd

data = pd.read_csv("hand_data.csv", header=None)
print(data.shape)
data.head()

(72, 64)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.508443,0.702552,-6.145772e-07,0.440408,0.715047,-0.063123,0.336590,0.617484,-0.087211,0.292894,...,0.484739,0.470087,-0.067160,0.496327,0.535629,-0.059018,0.500418,0.508773,-0.042466,Rock
1,0.469124,0.670497,-3.253003e-07,0.480272,0.584684,0.014141,0.469348,0.501550,0.010615,0.435712,...,0.318141,0.527720,-0.049043,0.327588,0.549460,-0.033742,0.341855,0.568334,-0.024701,Rock
2,0.495962,0.701034,-3.370984e-07,0.508619,0.611938,0.013030,0.490566,0.528399,0.006733,0.440792,...,0.335959,0.553930,-0.058957,0.345416,0.579337,-0.043023,0.361508,0.600703,-0.033410,Rock
3,0.490445,0.709333,-3.198429e-07,0.497623,0.620481,0.014015,0.474048,0.530295,0.008846,0.420658,...,0.329898,0.573723,-0.058713,0.340704,0.597497,-0.042168,0.356448,0.618607,-0.032982,Rock
4,0.519620,0.703897,-5.991295e-07,0.455413,0.684651,-0.033796,0.407203,0.576606,-0.044065,0.441917,...,0.537115,0.431677,-0.050166,0.532034,0.495520,-0.034640,0.535080,0.515030,-0.016664,Rock


In [36]:
# 열 이름 만들기
columns = []
for i in range(21):
    columns.extend([f"x{i}", f"y{i}", f"z{i}"])
columns.append("label")
data.columns = columns

In [37]:
data.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,label
0,0.508443,0.702552,-6.145772e-07,0.440408,0.715047,-0.063123,0.336590,0.617484,-0.087211,0.292894,...,0.484739,0.470087,-0.067160,0.496327,0.535629,-0.059018,0.500418,0.508773,-0.042466,Rock
1,0.469124,0.670497,-3.253003e-07,0.480272,0.584684,0.014141,0.469348,0.501550,0.010615,0.435712,...,0.318141,0.527720,-0.049043,0.327588,0.549460,-0.033742,0.341855,0.568334,-0.024701,Rock
2,0.495962,0.701034,-3.370984e-07,0.508619,0.611938,0.013030,0.490566,0.528399,0.006733,0.440792,...,0.335959,0.553930,-0.058957,0.345416,0.579337,-0.043023,0.361508,0.600703,-0.033410,Rock
3,0.490445,0.709333,-3.198429e-07,0.497623,0.620481,0.014015,0.474048,0.530295,0.008846,0.420658,...,0.329898,0.573723,-0.058713,0.340704,0.597497,-0.042168,0.356448,0.618607,-0.032982,Rock
4,0.519620,0.703897,-5.991295e-07,0.455413,0.684651,-0.033796,0.407203,0.576606,-0.044065,0.441917,...,0.537115,0.431677,-0.050166,0.532034,0.495520,-0.034640,0.535080,0.515030,-0.016664,Rock


In [38]:
data["label"].value_counts()

label
Rock        26
Paper       23
Scissors    23
Name: count, dtype: int64

In [39]:
def encoding(label):
    if label == "Rock":
        return 0
    elif label == "Scissors":
        return 1
    elif label == "Paper":
        return 2
    
data["label_enc"] = data["label"].apply(lambda x: encoding(x))

In [40]:
data.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,y18,z18,x19,y19,z19,x20,y20,z20,label,label_enc
0,0.508443,0.702552,-6.145772e-07,0.440408,0.715047,-0.063123,0.336590,0.617484,-0.087211,0.292894,...,0.470087,-0.067160,0.496327,0.535629,-0.059018,0.500418,0.508773,-0.042466,Rock,0
1,0.469124,0.670497,-3.253003e-07,0.480272,0.584684,0.014141,0.469348,0.501550,0.010615,0.435712,...,0.527720,-0.049043,0.327588,0.549460,-0.033742,0.341855,0.568334,-0.024701,Rock,0
2,0.495962,0.701034,-3.370984e-07,0.508619,0.611938,0.013030,0.490566,0.528399,0.006733,0.440792,...,0.553930,-0.058957,0.345416,0.579337,-0.043023,0.361508,0.600703,-0.033410,Rock,0
3,0.490445,0.709333,-3.198429e-07,0.497623,0.620481,0.014015,0.474048,0.530295,0.008846,0.420658,...,0.573723,-0.058713,0.340704,0.597497,-0.042168,0.356448,0.618607,-0.032982,Rock,0
4,0.519620,0.703897,-5.991295e-07,0.455413,0.684651,-0.033796,0.407203,0.576606,-0.044065,0.441917,...,0.431677,-0.050166,0.532034,0.495520,-0.034640,0.535080,0.515030,-0.016664,Rock,0


## 2) 데이터 분할하기

```
uv add scikit-learn
```

In [41]:
X = data.iloc[:, :-2]
y = data["label_enc"]
print(X.shape, y.shape)

(72, 63) (72,)


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")
 

X_train: (57, 63), y_train: (57,), X_test: (15, 63), y_test: (15,)


## 3) 모델 만들기

```
uv add xgboost
```

In [43]:
from xgboost import XGBClassifier
import numpy as np

model = XGBClassifier(n_estimator=10, random_state=42)
model.fit(np.array(X_train), np.array(y_train))

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [44]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.50      0.67      0.57         3
           2       0.83      0.83      0.83         6

    accuracy                           0.80        15
   macro avg       0.78      0.78      0.77        15
weighted avg       0.83      0.80      0.81        15



In [45]:
import joblib

joblib.dump(model, "model/rock_scissors_paper.pkl")

['model/rock_scissors_paper.pkl']

In [46]:
import joblib
model = joblib.load("model/rock_scissors_paper.pkl")

In [47]:
model

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
